In [1]:
from package import *
import requests
from pprint import pprint

class CallApi:
    endpoints = {
        'videos': 'https://www.googleapis.com/youtube/v3/videos',
        'searchs': 'https://www.googleapis.com/youtube/v3/search',
        'channels': 'https://www.googleapis.com/youtube/v3/channels',
        'playlists': 'https://www.googleapis.com/youtube/v3/playlists',
        'playlistItems': 'https://www.googleapis.com/youtube/v3/playlistItems',
        'activities': 'https://www.googleapis.com/youtube/v3/activities',
        'watch': 'http://www.youtube.com/watch'
    }
    def __init__(self, **kwargs):
        self.api_key = github.youtube_api_key
        
    def request(self, endpoint, **parameters):
        if not endpoint.endswith('s'): endpoint += 's'
        url = self.endpoints[endpoint]

        parameters['key'] = self.api_key

        response = requests.get(url, params = parameters)
        return response
test_api = CallApi()
youtube = YouTube(api_key = github.youtube_api_key)

playlist_parameters = {
    'id': "PL1cXh4tWqmsEQPeLEJ5V3k5knt-X9k043",
    'maxResults': 50,
    'part': "snippet,contentDetails",
}
search_parameters = {
    'playlistId': playlist_parameters['id'],
    'maxResults': 50,
    'part': 'snippet',
}
channel_parameters = {
    'id':'UCkxctb0jr8vwa4Do6c6su0Q',
    'part': "snippet,statistics,topicDetails"
}
video_parameters = {
    'id': 'p9RKAalhtZM',
    'part': 'snippet,statistics,topicDetails'
}

response = test_api.request('videos', **video_parameters)

pprint(response.json(), width = 200)
print(type(response))
print("Finished!")

{'etag': '"S8kisgyDEblalhHF9ooXPiFFrkc/fQE1kmxiraLiUvcYPKOBP5ywKGY"',
 'items': [{'etag': '"S8kisgyDEblalhHF9ooXPiFFrkc/Bz32e1_wjpvbltSAB82giacaHGw"',
            'id': 'p9RKAalhtZM',
            'kind': 'youtube#video',
            'snippet': {'categoryId': '20',
                        'channelId': 'UCboMX_UNgaPBsUOIgasn3-Q',
                        'channelTitle': 'Funhaus',
                        'description': 'Watch all of our back catalog 24/7 together as a community (so you can make fun of us), and keep your eyes peeled for special surprises throughout the stream '
                                       '(just little bumpers we shot). \n'
                                       '\n'
                                       'Eventually, we want to turn this into something where users can vote what to watch next AND we want to be able to cut in with brand new live content as well.\n'
                                       '\n'
                                       'Enjoy everyone!

In [2]:
result = youtube.get('videos', video_parameters['id'])
pprint(result.toStandard(), width = 200)

{'channelId': 'UCboMX_UNgaPBsUOIgasn3-Q',
 'itemId': 'p9RKAalhtZM',
 'itemKind': 'video',
 'videoDescription': 'Watch all of our back catalog 24/7 together as a community (so you can make fun of us), and keep your eyes peeled for special surprises throughout the stream (just little '
                     'bumpers we shot). \n'
                     '\n'
                     'Eventually, we want to turn this into something where users can vote what to watch next AND we want to be able to cut in with brand new live content as well.\n'
                     '\n'
                     'Enjoy everyone!',
 'videoDislikeCount': 445,
 'videoDuration': Duration(0),
 'videoId': 'p9RKAalhtZM',
 'videoLikeCount': 11937,
 'videoName': 'Funhaus TV! (check the description)',
 'videoPublishDate': Timestamp(2017, 12, 18, 23, 5, 6),
 'videoTags': ['funhaus', 'live'],
 'videoViewCount': 1084089}
